In [94]:
# general imports
import pandas as pd
import numpy as np
import capstone_utility_functions as cuf
import seaborn as sns
import matplotlib.pyplot as plt

# webscraping imports
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

# misc imports
import joblib
import re
import io 
from PyPDF2 import PdfReader
import pdftotext

# NLP imports
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk.corpus
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string

# LDA imports
import gensim 
import pyLDAvis
from gensim import corpora
import pyLDAvis.gensim_models
from sklearn.decomposition import LatentDirichletAllocation
pyLDAvis.enable_notebook()

/Users/henryparemain/opt/anaconda3/envs/Capstone/lib/python3.7/site-packages/plac_ext.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [182]:
# /v2/everything
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='668cf5020e5840668d4b18b450db8e40')

# from newsapi import NewsApiClient

all_articles = newsapi.get_everything(q='Apple',
                                    #   sources='bbc-news,the-verge',
                                      # domains='bbc.co.uk,techcrunch.com',
                                      from_param='2023-04-01',
                                      language='en',
                                      sort_by='relevancy')
                                      # page=2)

In [180]:
# # /v2/top-headlines
# top_headlines = newsapi.get_top_headlines(q='Apple',
#                                         #   sources='bbc-news,the-verge',
#                                           category='business')
#                                           # language='en')
#                                         #   country='us')

In [185]:
all_articles

{'status': 'ok',
 'totalResults': 0,
 'articles': [{'title': 'The Apple Watch SE drops back to $219',
   'author': 'Jeff Dunn',
   'source': {'Id': 'engadget', 'Name': 'Engadget'},
   'publishedAt': '2023-04-03T14:40:32Z',
   'url': 'https://www.engadget.com/the-apple-watch-se-drops-back-to-219-144032655.html'},
  {'title': 'Apple TV 4K is getting multiview for sports streams',
   'author': 'Emma Roth',
   'source': {'Id': 'the-verge', 'Name': 'The Verge'},
   'publishedAt': '2023-04-14T21:55:07Z',
   'url': 'https://www.theverge.com/2023/4/14/23683917/apple-tv-4k-multiview-sports-baseball-soccer'},
  {'title': 'Apple reportedly held anti-union meetings at all of its US stores',
   'author': 'Igor Bonifacic',
   'source': {'Id': 'engadget', 'Name': 'Engadget'},
   'publishedAt': '2023-04-09T22:35:28Z',
   'url': 'https://www.engadget.com/apple-reportedly-held-anti-union-meetings-at-all-of-its-us-stores-223528059.html'},
  {'title': 'How to Use Karaoke Mode (2023): Apple Music, Spotify,

In [184]:
import joblib
ldamodel = joblib.load('../data/3_topics_1200_reports_lda_model.pkl')

In [96]:
df = joblib.load('../data/clean_paras_df.pkl')

In [97]:
dictionary = joblib.load('../data/trained_lda_1200_dictionary.pkl')
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df.clean_paras]

In [102]:
topic_data =  pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, mds = 'pcoa')
all_topics = {}
num_terms = 20 # Adjust number of words to represent each topic
lambd = 0 # Adjust this accordingly based on tuning above
for i in range(1,3+1): #Adjust this to reflect number of topics chosen for final LDA model
    topic = topic_data.topic_info[topic_data.topic_info.Category == 'Topic'+str(i)].copy()
    topic['relevance'] = topic['loglift']*(1-lambd)+topic['logprob']*lambd
    all_topics['Topic '+str(i)] = topic.sort_values(by='relevance', ascending=False).Term[:num_terms].values
key_topics_df = pd.DataFrame(all_topics).T

In [103]:
key_topics_df.T

,Topic 1,Topic 2,Topic 3
0,community,risk,emission
1,family,security,product
2,people,audit,water
3,fund,ethic,waste
4,student,disclosure,carbon
5,school,code,scope
6,gender,statement,energy
7,care,assessment,intensity
8,career,conduct,tonne
9,child,page,biodiversity


We need to get all 60 words in a list in the above dataframe so that they can be used to pick out relevant headlines.

In [111]:
(key_topics_df.values)[0]

array(['community', 'family', 'people', 'fund', 'student', 'school',
       'gender', 'care', 'career', 'child', 'workforce', 'talent',
       'skill', 'foundation', 'colleague', 'equity', 'experience',
       'education', 'woman', 'inclusion'], dtype=object)

In [112]:
li = []
for i in key_topics_df.values:
    for j in i:
        li.append(j)

In [114]:
len(li)

60

In [138]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
final_bank = []
for word in li:
    # Replace this with the lemmatized word you want to generate inflections for
    og_word = word

    # Determine the part of speech of the lemmatized word
    pos = wordnet.synsets(lemmatized_word)[0].pos()

    # Create a WordNet lemmatizer and use it to lemmatize the word again
    lemmatizer = WordNetLemmatizer()
    lemmatized_word = lemmatizer.lemmatize(og_word, pos)

    # Use NLTK's inflection module to generate possible inflections of the word
    from nltk.corpus import wordnet
    from nltk.stem import WordNetLemmatizer
    from nltk.stem import SnowballStemmer
    from nltk import pos_tag
    from nltk import word_tokenize
    from nltk import sent_tokenize
    from nltk.corpus import stopwords
    from nltk.corpus import wordnet
    from nltk.stem import PorterStemmer
    from nltk.stem import LancasterStemmer
    from nltk.stem import SnowballStemmer

    stemmer = SnowballStemmer("english")
    stemmed_word = stemmer.stem(lemmatized_word)
    all_words = []
    # Print the generated inflections
    for syn in wordnet.synsets(lemmatized_word):
        for lemma in syn.lemmas():
            if stemmed_word in lemma.name():
                all_words.append(lemma.name())
    all_words = [x for x in all_words if x!=og_word]
    all_words.append(og_word)
    final_bank.append(all_words)

In [140]:
words = []
for x in final_bank:
    for y in x:
        words.append(y)

In [142]:
len(words)

140

In [143]:
words = [x.replace('_',' ') for x in words]

In [144]:
words

['community',
 'family',
 'people',
 'monetary fund',
 'fund',
 'student',
 'schoolhouse',
 'schooling',
 'schooltime',
 'school day',
 'school',
 'grammatical gender',
 'gender',
 'give care',
 'care',
 'career',
 'child',
 'workforce',
 'talent',
 'skill',
 'foundation garment',
 'foundation',
 'colleague',
 'equity',
 'experience',
 'educational activity',
 'education',
 'charwoman',
 'cleaning woman',
 'womanhood',
 'woman',
 'inclusion body',
 'cellular inclusion',
 'inclusion',
 'risk of infection',
 'risk of exposure',
 'run a risk',
 'risk',
 'security department',
 'security system',
 'security measure',
 'security measures',
 'security',
 'audited account',
 'audit',
 'ethical code',
 'ethic',
 'disclosure',
 'computer code',
 'write in code',
 'code',
 'financial statement',
 'statement',
 'assessment',
 'conduct',
 'pageboy',
 'page',
 'right field',
 'rightfield',
 'right wing',
 'right hand',
 'rightfulness',
 'right-hand',
 'the right way',
 'right on',
 'aright',
 'righ

we now have a list of words which are indicative of esg. We can try to select only the headlines which have one of these words in and see what we get.

In [190]:
url_li = []
for i in all_articles['articles']:
    for word in words:
        if word in i['title']:
            url_li.append(i['url'])
            break

In [191]:
url_li

['https://www.engadget.com/samsung-galaxy-watch-users-can-now-view-their-heart-rate-on-peloton-equipment-140056941.html',
 'https://consent.google.com/ml?continue=https://news.google.com/rss/articles/CBMihgFodHRwczovL3RpbWVzb2ZpbmRpYS5pbmRpYXRpbWVzLmNvbS9nYWRnZXRzLW5ld3MvYXBwbGUtdG8tdXNlLW1vcmUtcmVjeWNsZWQtbWF0ZXJpYWxzLWluLWl0cy1wcm9kdWN0cy1ieS0yMDI1L2FydGljbGVzaG93Lzk5NDcyNTE3LmNtc9IBgQFodHRwczovL20udGltZXNvZmluZGlhLmNvbS9nYWRnZXRzLW5ld3MvYXBwbGUtdG8tdXNlLW1vcmUtcmVjeWNsZWQtbWF0ZXJpYWxzLWluLWl0cy1wcm9kdWN0cy1ieS0yMDI1L2FtcF9hcnRpY2xlc2hvdy85OTQ3MjUxNy5jbXM?oc%3D5&gl=FR&hl=en-US&cm=2&pc=n&src=1',
 'https://www.androidcentral.com/phones/google-find-my-device-phone-off-rumor',
 'https://gizmodo.com/tiktok-ban-tik-tok-ceo-cybersecurity-security-disaster-1850334082']

In [ ]:
# INGESTION THROUGH WEB SCRAPING USING BEAUTIFULSOUP
def ingest_google_news():
    ticker_list = ['AAPL.O', 'LVS', 'COTY.K','JPM', 'XOM', '005930.KS']

    sep = '.'
    
    df = pd.DataFrame()
    t_news = []
    t_publisher = []
    t_urls = []
    t_dates = []
    t_tickers = []

    for t in ticker_list:
        news = []
        publisher = []
        urls = []
        dates = []
        tickers = []

        # cleaning ticker
        ticker = t
        t = t.split(sep, 1)[0]

        # set header by random user agent 
        r = requests.Session()
        headers = random_header()
        r.headers = headers
        # print(headers)

        # set query for google
        query = '{} stock news'.format(t)
        url = f"https://www.google.com/search?q={query}&tbm=nws&lr=lang_en&hl=en&sort=date&num=5"
        res = r.get(url, headers=headers)
        soup = bs4.BeautifulSoup(res.text, "html.parser")
        
        links = soup.select(".dbsr a")
        for l in links:
            tickers.append(t)
            try:
                url_w = l.get("href")
                print(url_w)
                urls.append(url_w)
                dt = find_date(url_w)
                dates.append(dt)

                res = requests.get(url_w, headers=headers)
                parsed_article = bs4.BeautifulSoup(res.text,'lxml')
                paragraphs = parsed_article.find_all('p')

                article_text = ""
                for p in paragraphs:
                    article_text += p.text

            except Exception as e:
                article_text = ''

            news.append(article_text)

        sources = soup.select(".XTjFC g-img")
        for s in sources:
            publisher.append(s.next_sibling.lower())

        t_urls += urls
        t_news += news
        t_publisher += publisher
        t_dates += dates
        t_tickers += tickers

    df['ticker'] = t_tickers
    df['links'] = t_urls
    df['article_text'] = t_news
    df['publisher'] = t_publisher
    df['created_at'] = t_dates

    # import to csv
    today = date.today()
    d1 = today.strftime("%d%m%Y")
    df.to_csv(f'data_news/google_news_{d1}.csv')

    del news, publisher, urls, dates, tickers
    del t_news, t_publisher, t_urls, t_dates, t_ticker

In [117]:
print(response.json)

<bound method Response.json of <Response [200]>>


In [69]:
response.json()['articles'][0]['url']

'https://www.engadget.com/8bitdo-controllers-now-work-with-apple-devices-163657954.html'

In [70]:
li = []
for i in list(range(0,len(response.json()['articles']))):
    li.append(response.json()['articles'][i]['url'])


In [71]:
li

['https://www.engadget.com/8bitdo-controllers-now-work-with-apple-devices-163657954.html',
 'https://www.wired.com/story/tetris-movie-review/',
 'https://www.engadget.com/apple-accused-of-illegally-firing-pro-union-workers-140058541.html',
 'https://www.engadget.com/the-apple-watch-se-drops-back-to-219-144032655.html',
 'https://www.wired.com/story/best-macbook-accessories/',
 'https://www.engadget.com/apple-music-bug-on-ios-is-reportedly-mixing-up-peoples-playlists-073003381.html',
 'https://www.theverge.com/2023/4/14/23683917/apple-tv-4k-multiview-sports-baseball-soccer',
 'https://www.engadget.com/apple-reportedly-held-anti-union-meetings-at-all-of-its-us-stores-223528059.html',
 'https://www.wired.com/story/how-to-sing-along-apple-music-spotify/',
 'https://www.wired.com/story/electric-cars-qualcomm-apple/',
 'https://www.engadget.com/ios-164-is-out-with-bug-fixes-and-a-ton-of-new-emoji-181331049.html',
 'https://www.engadget.com/apples-weather-app-is-broken-for-many-users-13540919

In [82]:
import requests
from bs4 import BeautifulSoup

# Replace this with the URL of the webpage you want to extract text from
url = 'https://www.engadget.com/8bitdo-controllers-now-work-with-apple-devices-163657954.html'

# Make a GET request to the webpage
response = requests.get(url)

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the text from the HTML content
text = soup.get_text()

# Print the extracted text
text

'8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGearE

In [83]:
import re
modified_text = re.sub(r'\n{2,}', '\n\n', text)


In [84]:
modified_text = re.sub(r'\t{1,}', '\n\n', modified_text)


In [85]:
modified_text = re.sub(r'\n{2,}', '\n\n', modified_text)


In [86]:
modified_text

'8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGearE

In [87]:
modified_text = modified_text.split('\n\n')

In [88]:
modified_text

['8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGear

In [89]:
modified_text = [x.replace('\n',' ') for x in modified_text]

In [90]:
modified_text

['8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGear

In [91]:
res = [x for x in modified_text if len(x.split())>10]
res

['8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGear

In [223]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
import re
import random

def scrape_google_news(ticker_list, num_articles=5):
    df = pd.DataFrame()
    headers_list = [{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}, 
                    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'},
                    {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'},
                    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'},
                    {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'}]
    sep = '.'

    for ticker in ticker_list:
        ticker_short = ticker.split(sep, 1)[0]
        url = f"https://www.google.com/search?q={ticker_short} news&tbm=nws&lr=lang_en&hl=en&sort=date&num={num_articles}"
        headers = random.choice(headers_list)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        links = soup.select('.dbsr a')
        urls = []
        titles = []
        for link in links:
            url = link.get('href')
            if 'url?q=' in url:
                url = url.split('url?q=')[1].split('&sa=')[0]
            else:
                continue
            if 'webcache' in url:
                continue
            urls.append(url)
            title = link.text
            titles.append(title)

        publishers = []
        dates = []
        for url in urls:
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            try:
                publisher = soup.find('a', class_='source').text
            except:
                publisher = ''
            publishers.append(publisher)
            try:
                date = soup.find('time').get('datetime')
            except:
                date = ''
            dates.append(date)

        news_df = pd.DataFrame({'ticker': ticker,
                                'title': titles,
                                'publisher': publishers,
                                'url': urls,
                                'date': dates})
        df = pd.concat([df, news_df], ignore_index=True)

    today = date.today().strftime('%Y-%m-%d')
    df.to_csv(f'google_news_{today}.csv', index=False)
    return df


In [224]:
ticker_list = ['AAPL.O', 'AMZN', 'GOOGL', 'MSFT', 'NVDA', 'TSLA', 'JPM', 'BAC', 'WFC', 'V', 'MA', 'NFLX', 'FB', 'TWTR', 'UBER', 'LYFT', 'DIS', 'BA', 'LUV', 'DAL']

df = ingest_google_news(ticker_list)

In [225]:
df

,ticker,links,article_text,publisher,created_at


In [284]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=energy_transportation&time_from=20200401T0130&limit=20&apikey=CRHNI4SXD7JPGBC8'
r = requests.get(url)
data = r.json()
print(data)

{'items': '50', 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish', 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.', 'feed': [{'title': "United Airlines Shares Are Turbulent After Hours: What's Going On? - United Airlines Holdings  ( NASDAQ:UAL ) ", 'url': 'https://www.benzinga.com/news/earnings/23/04/31869613/united-airlines-shares-are-turbulent-after-hours-whats-going-on', 'time_published': '20230418T201658', 'authors': ['Adam Eckert'], 'summary': "United Airlines Holdings Inc UAL shares are volatile in Tuesday's after-hours session after the airline company reported first-quarter financial results. What Happened: United reported first-quarter revenue of $11.43 billion, which beat average analyst estimates of $11.42 billion, according to ...", 'banner_image': 'https://cdn.benzinga.com/files/images/story/2023/04/18/scree

In [285]:
data['feed'][0]['url']

'https://www.benzinga.com/news/earnings/23/04/31869613/united-airlines-shares-are-turbulent-after-hours-whats-going-on'

In [241]:
len(data['feed'])

50

In [286]:
urls = []
for i in list(range(0,len(data['feed']))):
    urls.append(data['feed'][i]['url'])

In [287]:
urls

['https://www.benzinga.com/news/earnings/23/04/31869613/united-airlines-shares-are-turbulent-after-hours-whats-going-on',
 'https://www.benzinga.com/pressreleases/23/04/g31869469/eneti-inc-to-announce-first-quarter-2023-results-and-have-a-conference-call-on-april-27-2023',
 'https://www.globenewswire.com/news-release/2023/04/18/2649568/0/en/Eneti-Inc-to-Announce-First-Quarter-2023-Results-and-Have-a-Conference-Call-on-April-27-2023.html',
 'https://www.benzinga.com/pressreleases/23/04/g31869351/daseke-inc-to-release-first-quarter-2023-results-on-may-9-2023',
 'https://www.globenewswire.com/news-release/2023/04/18/2649564/0/en/Daseke-Inc-to-Release-First-Quarter-2023-Results-on-May-9-2023.html',
 'https://www.reuters.com/business/aerospace-defense/united-airlines-reports-smaller-than-expected-quarterly-loss-2023-04-18/',
 'https://www.cnbc.com/2023/04/18/united-airlines-ual-earnings-q1-2023.html',
 'https://www.reuters.com/markets/deals/permian-spotlight-energy-dealmaking-gathers-steam-

In [244]:
import requests
from bs4 import BeautifulSoup

# Replace this with the URL of the webpage you want to extract text from
url = 'https://www.benzinga.com/news/22/11/29901041/musks-twitter-war-on-apple-acts-as-fodder-for-spotify-and-epic-games-fight-against-iphone-maker'

# Make a GET request to the webpage
response = requests.get(url)

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the text from the HTML content
text = soup.get_text()
# Print the extracted text
text

'Musk\'s Twitter War On Apple Acts As Fodder For Spotify And Epic Games\' Fight Against iPhone Maker - Apple (NASDAQ:AAPL), Spotify Technology (NYSE:SPOT) - BenzingaEspañolIndiaItalianoFrançaisMy AccountMy AccountNotificationsOverview+ New WatchlistBenzinga ResearchBenzinga ProLog In––%––%––%––%––%––%––%Data & APIsEventsMarketfyPremarketBoostContributeEspañolIndiaItalianoFrançaisSign inOur Services News EarningsGuidanceDividendsM&ABuybacksLegalInterviewsManagementOfferingsIPOsInsider TradesBiotech/FDAFreightPoliticsGovernmentHealthcareMarkets Pre-MarketAfter HoursMoversETFsForexCannabisCommoditiesOptionsBinary OptionsBondsFuturesCME GroupGlobal EconomicsPreviewsSmall-CapCryptocurrencyPenny StocksDigital SecuritiesVolatilityRatings Analyst ColorDowngradesUpgradesInitiationsPrice TargetIdeas Trade IdeasCovey Trade IdeasLong IdeasShort IdeasTechnicalsFrom The PressJim CramerRumorsBest Stocks & ETFsBest Penny StocksBest S&P 500 ETFsBest Swing Trade StocksBest Blue Chip StocksBest High-Volu

In [245]:
text.split('\n')

["Musk's Twitter War On Apple Acts As Fodder For Spotify And Epic Games' Fight Against iPhone Maker - Apple (NASDAQ:AAPL), Spotify Technology (NYSE:SPOT) - BenzingaEspañolIndiaItalianoFrançaisMy AccountMy AccountNotificationsOverview+ New WatchlistBenzinga ResearchBenzinga ProLog In––%––%––%––%––%––%––%Data & APIsEventsMarketfyPremarketBoostContributeEspañolIndiaItalianoFrançaisSign inOur Services News EarningsGuidanceDividendsM&ABuybacksLegalInterviewsManagementOfferingsIPOsInsider TradesBiotech/FDAFreightPoliticsGovernmentHealthcareMarkets Pre-MarketAfter HoursMoversETFsForexCannabisCommoditiesOptionsBinary OptionsBondsFuturesCME GroupGlobal EconomicsPreviewsSmall-CapCryptocurrencyPenny StocksDigital SecuritiesVolatilityRatings Analyst ColorDowngradesUpgradesInitiationsPrice TargetIdeas Trade IdeasCovey Trade IdeasLong IdeasShort IdeasTechnicalsFrom The PressJim CramerRumorsBest Stocks & ETFsBest Penny StocksBest S&P 500 ETFsBest Swing Trade StocksBest Blue Chip StocksBest High-Volum

Lets try and iterate through all 50 urls and split each webpage into paragraphs. See what we end up with.

In [288]:
import requests
from bs4 import BeautifulSoup
para_bank = []

for url in urls:
    # Replace this with the URL of the webpage you want to extract text from
    url = url
    # Make a GET request to the webpage
    response = requests.get(url)

    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the text from the HTML content
    text = soup.get_text()
    # Print the extracted text
    text = text.split('\n')
    para_bank.extend(text)

In [289]:
para_bank

['United Airlines Shares Are Turbulent After Hours: What\'s Going On? - United Airlines Holdings (NASDAQ:UAL) - BenzingaEspañolIndiaItalianoFrançaisMy AccountMy AccountNotificationsOverview+ New WatchlistBenzinga ResearchBenzinga ProLog In––%––%––%––%––%––%––%Data & APIsEventsMarketfyPremarketBoostContributeEspañolIndiaItalianoFrançaisSign inOur Services News EarningsGuidanceDividendsM&ABuybacksLegalInterviewsManagementOfferingsIPOsInsider TradesBiotech/FDAFreightPoliticsGovernmentHealthcareMarkets Pre-MarketAfter HoursMoversETFsForexCannabisCommoditiesOptionsBinary OptionsBondsFuturesCME GroupGlobal EconomicsPreviewsSmall-CapCryptocurrencyPenny StocksDigital SecuritiesVolatilityRatings Analyst ColorDowngradesUpgradesInitiationsPrice TargetIdeas Trade IdeasCovey Trade IdeasLong IdeasShort IdeasTechnicalsFrom The PressJim CramerRumorsBest Stocks & ETFsBest Penny StocksBest S&P 500 ETFsBest Swing Trade StocksBest Blue Chip StocksBest High-Volume Penny StocksBest Small Cap ETFsBest Stocks

Lets remove any string which is empty, or less than 10 words long.

In [290]:
updated_para_bank = []
for sent in para_bank:
    if len(sent.split())<10 or sent == '':
        continue
    else:
        updated_para_bank.append(sent)

In [291]:
updated_para_bank

['United Airlines Shares Are Turbulent After Hours: What\'s Going On? - United Airlines Holdings (NASDAQ:UAL) - BenzingaEspañolIndiaItalianoFrançaisMy AccountMy AccountNotificationsOverview+ New WatchlistBenzinga ResearchBenzinga ProLog In––%––%––%––%––%––%––%Data & APIsEventsMarketfyPremarketBoostContributeEspañolIndiaItalianoFrançaisSign inOur Services News EarningsGuidanceDividendsM&ABuybacksLegalInterviewsManagementOfferingsIPOsInsider TradesBiotech/FDAFreightPoliticsGovernmentHealthcareMarkets Pre-MarketAfter HoursMoversETFsForexCannabisCommoditiesOptionsBinary OptionsBondsFuturesCME GroupGlobal EconomicsPreviewsSmall-CapCryptocurrencyPenny StocksDigital SecuritiesVolatilityRatings Analyst ColorDowngradesUpgradesInitiationsPrice TargetIdeas Trade IdeasCovey Trade IdeasLong IdeasShort IdeasTechnicalsFrom The PressJim CramerRumorsBest Stocks & ETFsBest Penny StocksBest S&P 500 ETFsBest Swing Trade StocksBest Blue Chip StocksBest High-Volume Penny StocksBest Small Cap ETFsBest Stocks

In [292]:
len(updated_para_bank)

407

We have 603 paragraphs. These may or may not have anything of use in. Could be interesting to feed into LDA model and see if it can identify most relevant paragraph.

In [293]:
df = pd.DataFrame(data=updated_para_bank)
df.rename(columns={0:'raw_paras'},inplace=True)

In [294]:
# list of noisy words identified through the iterative process
noisy = ['clients','west','percent','letter','sustainability','report','appendix','management','team','social','corporate','responsibility','business','pages','introduction']

# feed the raw paragraphs into the custom cleaner. Note that tokens less than 3 letters long are treated as noise and removed.
df['clean_paras'] = [cuf.my_cleaner(x,3,noisy_words = noisy, newLineRemove = True,lemmatization=True,nouns = True,) for x in df.raw_paras]
df

,raw_paras,clean_paras
0,United Airlines Shares Are Turbulent After Hou...,"[airline, share, hour, airline, holding, benzi..."
1,United will hold a conference call to discuss ...,"[hold, conference, call, discus, result, airli..."
2,Eneti Inc. to Announce First Quarter 2023 Resu...,"[eneti, announce, quarter, result, conference,..."
3,Eneti Inc. is a leading provider of installati...,"[eneti, provider, installation, maintenance, v..."
4,Eneti Inc. to Announce First Quarter 2023 Resu...,"[eneti, announce, quarter, result]"
...,...,...
402,WTI Crude Oil (/CL) fell slightly to $80.56 pe...,"[crude, barrel]"
403,"The 10-year Treasury note yield (TNX), a bench...","[year, treasury, yield, benchmark, indicator, ..."
404,CHART OF THE DAY: 10-YEAR TREMORS. The 10-year...,"[chart, year, tremor, year, treasury, yield, c..."
405,Thinking capIdeas to mull as you trade or inve...,"[idea, trade, invest, june, analyst, calculati..."


In [295]:
from sklearn.preprocessing import MinMaxScaler
# load in pretrained lda model for 3 topics
lda_model = joblib.load('../data/3_topics_1200_reports_lda_model.pkl')
#initialise empty dataframe to store probabilities in
a = np.empty((df.shape[0],3))
a[:] = np.nan
df_probs = pd.DataFrame(columns=['S','G','E'], data =a)
# iterate through clean paras and score the paragraph
for index, row in df.iterrows():
    #get the topic probabilities
    test_bow = lda_model.id2word.doc2bow(df.clean_paras[index])
    res = lda_model[test_bow]
    #append probabilities to the df_probs
    for i in res:
        row_num = index
        col_num = i[0]
        probability = i[1]
        df_probs.iloc[row_num, col_num] = probability
df_probs = pd.concat([df[['raw_paras','clean_paras']],df_probs],axis=1)
# df_probs = df_probs[df_probs['clean_paras'].apply(lambda x: len(x))>5]
# df_scores = df_probs[['S','G','E']]
# #drop any missing values to be safe
# df_scores = df_scores.dropna()
# #identify threshold for outliers
# p99_e = np.percentile(df_scores['E'],99)
# # remove outliers
# df_scores_outliers_rem = df_scores[df_scores["E"] <= p99_e]
# #identify threshold for outliers
# p99_s = np.percentile(df_scores['S'], 99)
# # remove outliers
# df_scores_outliers_rem = df_scores_outliers_rem[df_scores_outliers_rem["S"] <= p99_s]
# #identify threshold for outliers
# p99_g = np.percentile(df_scores['G'], 99)
# # remove outliers
# df_scores_outliers_rem = df_scores_outliers_rem[df_scores_outliers_rem["G"] <= p99_g]
# # applying min_max scaler to scores
# scaler = MinMaxScaler()
# df_scores_fin= scaler.fit_transform(df_scores_outliers_rem)
# df_scores_fin = pd.DataFrame(df_scores_fin,columns=df_scores.columns)

In [296]:
df_probs

,raw_paras,clean_paras,S,G,E
0,United Airlines Shares Are Turbulent After Hou...,"[airline, share, hour, airline, holding, benzi...",0.472053,0.263912,0.264035
1,United will hold a conference call to discuss ...,"[hold, conference, call, discus, result, airli...",0.565490,0.232716,0.201794
2,Eneti Inc. to Announce First Quarter 2023 Resu...,"[eneti, announce, quarter, result, conference,...",0.519770,0.270346,0.209884
3,Eneti Inc. is a leading provider of installati...,"[eneti, provider, installation, maintenance, v...",0.102817,0.658424,0.238759
4,Eneti Inc. to Announce First Quarter 2023 Resu...,"[eneti, announce, quarter, result]",0.098113,0.124162,0.777726
...,...,...,...,...,...
402,WTI Crude Oil (/CL) fell slightly to $80.56 pe...,"[crude, barrel]",0.111232,0.111232,0.777535
403,"The 10-year Treasury note yield (TNX), a bench...","[year, treasury, yield, benchmark, indicator, ...",0.460642,0.220283,0.319074
404,CHART OF THE DAY: 10-YEAR TREMORS. The 10-year...,"[chart, year, tremor, year, treasury, yield, c...",0.188485,0.317574,0.493941
405,Thinking capIdeas to mull as you trade or inve...,"[idea, trade, invest, june, analyst, calculati...",0.521281,0.228492,0.250226


In [307]:
df_probs.sort_values('E',ascending=False).iloc[0,0]

'NOVA provides a comprehensive energy service package that includes electricity, operations and maintenance, monitoring, repairs, equipment upgrades, on-site power optimization, and diagnostics services. Its residential solar energy system fleet generates around 1,627 megawatts, catering to over 279,000 customers.'

This method seems to work quite well for Environmental paragraphs. Need to explore further the api and which topics are relevant for which task. Need to explore how we can narrow this down for a specific company.

In [ ]:
import re
modified_text = re.sub(r'\n{2,}', '\n\n', text)


In [ ]:
modified_text = re.sub(r'\t{1,}', '\n\n', modified_text)


In [ ]:
modified_text = re.sub(r'\n{2,}', '\n\n', modified_text)


In [ ]:
modified_text

'8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGearE

In [ ]:
modified_text = modified_text.split('\n\n')

In [ ]:
modified_text

['8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGear

In [ ]:
modified_text = [x.replace('\n',' ') for x in modified_text]

In [ ]:
modified_text

['8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGear

In [ ]:
res = [x for x in modified_text if len(x.split())>10]
res

['8BitDo controllers now work with Apple devices | EngadgetHomeReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsLoginSign upEngadgetLoginReviewsBest in TechHands-OnView all ReviewsBuying GuidesBest Wireless EarbudsBest Robot VacuumsBest LaptopsBest Gaming LaptopsBest VPNBest Bluetooth TrackersBest WebcamsView all Buying GuidesGamingBest GamesPlayStationNintendoXboxView all GamingGearAmazonAppleGoogleMicrosoftSamsungView all GearEntertainmentMoviesMusicTVYouTubeView all EntertainmentTomorrowSpaceAIRoboticsTransportationView all TomorrowDealsNewsVideoPodcastsFacebookTwitterYouTubeSectionsReviewsBuying GuidesGamingGear